### Ecuación de Evapotranspiración de Referencia (ET₀) – FAO 56 con subcomponentes

$
ET_0 = \frac{
0.408 \cdot \Delta \cdot R_n +
\gamma \cdot \frac{900}{T_{\text{mean}} + 273} \cdot u_2 \cdot (e_s - e_a)
}{
\Delta + \gamma \cdot (1 + 0.34 \cdot u_2)
}
$

---

**Subcomponentes:**

**1. Temperatura media:**

$
T_{\text{mean}} = \frac{T_{\text{max}} + T_{\text{min}}}{2}
$

**2. Presión de vapor de saturación:**

$
e_s = \frac{e_s(T_{\text{max}}) + e_s(T_{\text{min}})}{2}, \quad
e_s(T) = 0.6108 \cdot \exp\left(\frac{17.27 \cdot T}{T + 237.3}\right)
$

**3. Presión de vapor actual:**

$
e_a = e_s \cdot \frac{RH}{100}
$

**4. Pendiente de la curva de presión de vapor:**

$
\Delta = \frac{
4098 \cdot 0.6108 \cdot \exp\left(\frac{17.27 \cdot T_{\text{mean}}}{T_{\text{mean}} + 237.3}\right)
}{
(T_{\text{mean}} + 237.3)^2
}
$

**5. Presión atmosférica (a altitud \(z\)):**

$
P = 101.3 \cdot \left(\frac{293 - 0.0065 \cdot z}{293}\right)^{5.26}
$

**6. Constante psicrométrica:**

$
\gamma = 0.000665 \cdot P
$

**7. Radiación neta:**

$
R_n = R_{ns} - R_{nl}
$

$
R_{ns} = (1 - \alpha) \cdot R_s = 0.77 \cdot R_s
$

$
R_{nl} = \sigma \cdot \frac{(T_{\text{max}} + 273.16)^4 + (T_{\text{min}} + 273.16)^4}{2}
\cdot (0.34 - 0.14 \cdot \sqrt{e_a}) \cdot \left(1.35 \cdot \frac{R_s}{R_a} - 0.35\right)
$

**8. Radiación solar estimada:**

$
R_s = (0.16 + 0.00075 \cdot z) \cdot \sqrt{n} \cdot R_a
$

**9. Radiación extraterrestre (\(R_a\)):**

$
R_a = \frac{24 \cdot 60}{\pi} \cdot G_{sc} \cdot d_r \cdot 
\left[ \omega \cdot \sin(\phi) \cdot \sin(\delta) + 
\cos(\phi) \cdot \cos(\delta) \cdot \sin(\omega) \right]
$

Donde:

- $G_{sc} = 0.0820 \, \text{MJ} \cdot \text{m}^{-2} \cdot \text{min}^{-1} $
- $ d_r = 1 + 0.033 \cdot \cos\left(\frac{2\pi J}{365}\right) $
- $ \delta = 0.409 \cdot \sin\left(\frac{2\pi J}{365} - 1.39\right) $
- $ \omega = \arccos\left(-\tan(\phi) \cdot \tan(\delta)\right) $
- $ \phi $ = latitud en radianes
- $ J $ = día juliano
- $ \sigma = 4.903 \times 10^{-9} \, \text{MJ} \cdot \text{K}^{-4} \cdot \text{m}^{-2} \cdot \text{día}^{-1} $


In [1]:


import pandas as pd
import numpy as np

def leer_datos_despues_de_encabezado(ruta_csv):
    try:
        # Leer todo el archivo como líneas de texto
        with open(ruta_csv, 'r', encoding='utf-8') as f:
            lineas = f.readlines()

        # Buscar índice donde termina el encabezado
        indice_inicio_datos = None
        for i, linea in enumerate(lineas):
            if '-END HEADER-' in linea:
                indice_inicio_datos = i + 1  # La siguiente línea tiene los nombres de columnas
                break

        if indice_inicio_datos is None:
            raise ValueError("No se encontró el marcador '-END HEADER-' en el archivo.")

        # Cargar desde esa línea en adelante con pandas
        df = pd.read_csv(ruta_csv, skiprows=indice_inicio_datos)

        # Verificar que la columna DOY exista
        if 'DOY' not in df.columns:
            raise ValueError("La columna 'DOY' no se encuentra en los datos.")

        # Asegurar que DOY es numérica
        df['DOY'] = pd.to_numeric(df['DOY'], errors='coerce')
        df = df[df['DOY'].notna()].copy()
        df['DOY'] = df['DOY'].astype(int)

        # Obtener valores de inicio y fin
        doy_inicio = df['DOY'].min()
        doy_fin = df['DOY'].max()
        total_registros = len(df)

        # Reportar
        print("  ")
        print(f"Total de registros leídos: {total_registros}")
        print(f"Rango de DOY (día del año): {doy_inicio} a {doy_fin}")

        return df,total_registros

    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

    
def transformar_columnas(df_resultado):
    try:
        # Verificar que las columnas requeridas existan
        columnas_requeridas = {
            "DOY": "Día",
            "T2M_MAX": "Tmax",
            "T2M_MIN": "Tmin",
            "RH2M": "HR",
            "WS2M": "Ux",
            "ALLSKY_SFC_SW_DWN": "Rs",
            "PRECTOTCORR": "Pef"
        }

        for col in columnas_requeridas.keys():
            if col not in df_resultado.columns:
                raise KeyError(f"La columna '{col}' no se encuentra en el DataFrame original.")

        # Crear nuevo DataFrame con columnas renombradas
        data = df_resultado[list(columnas_requeridas.keys())].rename(columns=columnas_requeridas)

        print("Nuevo DataFrame creado exitosamente con las columnas requeridas:")
        
        return data

    except Exception as e:
        print(f"Error al transformar el DataFrame: {e}")
        return None   

    
    
##################
# Entrada de datos
###################

archivo = 'Metepec-FAO56-periodo.csv'  # Reemplaza por la ruta de archivo
df_resultado,n_dias = leer_datos_despues_de_encabezado(archivo)

# Día juliano
J = np.arange(1, n_dias + 1)

data = transformar_columnas(df_resultado)

# Constantes del sitio (puedes modificarlas)
z = 100        # altitud (m)
lat = 19.0     # latitud (grados)

# Temperatura media diaria
data['Tmean'] = (data['Tmax'] + data['Tmin']) / 2

# Presión de vapor de saturación
data['es'] = (0.6108 * np.exp((17.27 * data['Tmax']) / (data['Tmax'] + 237.3)) +
              0.6108 * np.exp((17.27 * data['Tmin']) / (data['Tmin'] + 237.3))) / 2

# Presión de vapor actual
data['ea'] = data['es'] * (data['HR'] / 100)

# Pendiente de curva de presión de vapor
data['delta'] = 4098 * (0.6108 * np.exp((17.27 * data['Tmean']) / (data['Tmean'] + 237.3))) / (data['Tmean'] + 237.3)**2

# Presión atmosférica
data['P'] = 101.3 * (((293 - 0.0065 * z) / 293) ** 5.26)

# Constante psicrométrica
data['gamma'] = 0.665e-3 * data['P']

# Radiación extraterrestre Ra
Gsc = 0.0820
dr = 1 + 0.033 * np.cos(2 * np.pi * J / 365)
delta_s = 0.409 * np.sin(2 * np.pi * J / 365 - 1.39)
phi = np.radians(lat)
omega = np.arccos(-np.tan(phi) * np.tan(delta_s))
Ra = (24 * 60 / np.pi) * Gsc * dr * (omega * np.sin(phi) * np.sin(delta_s) +
                                     np.cos(phi) * np.cos(delta_s) * np.sin(omega))

# La Radiación solar Rs de acuerdo con la literatura
# Se debe estimar usando datos Radiación superficial Sky surface Short wave 
# Pero en caso de tener el valor del número de horas de radiación n, se usa lo siguiente
#data['Rs'] = (0.16 + 0.00075 * z) * np.sqrt(data['n']) * Ra

# Radiación neta
sigma = 4.903e-9
data['Rns'] = 0.77 * data['Rs']
data['Rnl'] = (sigma * (((data['Tmax'] + 273.16)**4 + (data['Tmin'] + 273.16)**4) / 2) *
               (0.34 - 0.14 * np.sqrt(data['ea'])) * (1.35 * data['Rs'] / Ra - 0.35))
data['Rn'] = data['Rns'] - data['Rnl']

# Evapotranspiración de referencia (ET0) FAO56
data['ET0'] = (
    (0.408 * data['delta'] * data['Rn'] +
     data['gamma'] * (900 / (data['Tmean'] + 273)) * data['Ux'] * (data['es'] - data['ea'])) /
    (data['delta'] + data['gamma'] * (1 + 0.34 * data['Ux']))
)

# Asignación simple de Kc por etapas (puedes modificar por fechas reales)
# Se usaron 4 etapas y distintos valores de Kc=inicial=0.3, desarrollo=medio=1.2, tardío=0.36 para cada una de ellas
# Las etapas son del 25% del total de los datos
data['Kc'] = np.select(
    [J <= 10, (J > 10) & (J <= 20), J > 20],
    [0.3, 0.8, 1.1]
)
####################################################


# Calcular el número total de filas
n_dat = len(data)

# Calcular los índices de corte en bloques del 25%
q1 = int(n_dat * 0.25)
q2 = int(n_dat * 0.50)
q3 = int(n_dat * 0.75)

# Crear un array vacío
kc_values = np.empty(n_dat)

# Asignar valores por segmentos
kc_values[:q1] = 0.3         # Primer 25%
kc_values[q1:q2] = 1.2       # Segundo 25%
kc_values[q2:q3] = 1.2       # Tercer 25%
kc_values[q3:] = 0.36        # Último 25%

# Agregar al DataFrame como nueva columna
data['Kc'] = kc_values



####################################################

# ETc (ET0 * Kc)
data['ETc'] = data['ET0'] * data['Kc']

# Resultado final
print(data[['Tmax', 'Tmin', 'HR', 'Ux', 'Rs', 'ET0', 'Kc', 'ETc','Pef']].round(3))

# Asignar número de década: día 0-9 → decada 1, día 10-19 → decada 2, etc.
data['decada'] = (data.index // 10) + 1

# Agrupar por década y sumar ETc
resumen_decadas = data.groupby('decada')['ETc'].sum().reset_index()
resumen_decadas.rename(columns={'ETc': 'ETc_total_decada'}, inplace=True)

# Mostrar resultado
print("Se crearon las décadas a partir del archivo de datos: ")
print(resumen_decadas)
# Calculamos la evapotranspiración de Huella Verde y Azul Por Día
data['ETverde'] = np.minimum(data['ETc'], data['Pef'])
data['ETazul'] = np.maximum(0, data['ETc'] - data['ETverde'])

# Rendimiento del cultivo (cambiar según el caso real)
R = 7000  # kg/ha

# Sumas de ETverde y ETazul
ETverde_total = data['ETverde'].sum()  # mm
ETazul_total = data['ETazul'].sum()    # mm

# Conversión a m³/ha
UACverde = ETverde_total * 10
UACazul = ETazul_total * 10

# Cálculo de huella hídrica (m³/ton)
HHverde = UACverde / (R / 1000)
HHazul = UACazul / (R / 1000)

# Mostrar resultados
print("   ")
print("R E S U L T A D O S    G E N E R A D O S  ")
print(f"ETverde total (mm): {ETverde_total:.2f}")
print(f"ETazul total  (mm): {ETazul_total:.2f}")
print(f"Uso de agua verde (m³/ha): {UACverde:.2f}")
print(f"Uso de agua azul  (m³/ha): {UACazul:.2f}")
print(f"Huella Hídrica Verde (m³/ton): {HHverde:.2f}")
print(f"Huella Hídrica Azul  (m³/ton): {HHazul:.2f}")
print("    ")
arch_salida="datos_huella_hidrica_140725.xlsx"
data.to_excel(arch_salida, index=False)
print("Archivo de salida generado:",arch_salida)
data

ModuleNotFoundError: No module named 'pandas'

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(data) 

,Día,Tmax,Tmin,HR,Ux,Rs,Pef,Tmean,es,ea,delta,P,gamma,Rns,Rnl,Rn,ET0,Kc,ETc
0,101,26.64,9.51,51.00,0.99,23.74,0.99,18.075,2.339474,1.193132,0.130308,100.123508,0.066582,18.2798,5.728041,12.551759,4.108318,0.30,1.232495
1,102,26.28,11.11,47.17,0.99,22.58,0.48,18.695,2.369912,1.117888,0.134817,100.123508,0.066582,17.3866,5.501811,11.884789,4.058618,0.30,1.217586
2,103,28.13,10.74,41.26,0.89,23.66,0.00,19.435,2.549417,1.051890,0.140370,100.123508,0.066582,18.2182,6.069088,12.149112,4.266407,0.30,1.279922
3,104,28.73,11.51,38.94,0.77,21.31,0.00,20.120,2.650742,1.032199,0.145683,100.123508,0.066582,16.4087,5.289273,11.119427,3.986596,0.30,1.195979
4,105,28.71,12.21,45.43,1.18,21.28,0.83,20.460,2.680569,1.217783,0.148382,100.123508,0.066582,16.3856,4.957046,11.428554,4.321252,0.30,1.296376
5,106,30.06,12.74,38.87,1.25,25.24,0.07,21.400,2.865088,1.113660,0.156067,100.123508,0.066582,19.4348,6.619684,12.815116,5.027473,0.30,1.508242
6,107,31.07,11.70,33.66,1.34,27.50,0.00,21.385,2.942793,0.990544,0.155941,100.123508,0.066582,21.1750,7.746068,13.428932,5.484927,0.30,1.645478
7,108,29.32,11.95,40.34,1.05,24.05,0.86,20.635,2.739149,1.104973,0.149788,100.123508,0.066582,18.5185,6.104643,12.413857,4.617414,0.30,1.385224
8,109,28.07,10.35,57.29,0.83,21.64,4.13,19.210,2.526213,1.447267,0.138662,100.123508,0.066582,16.6628,4.560968,12.101832,3.875741,0.30,1.162722
9,110,27.49,9.62,50.17,1.21,25.24,1.49,18.555,2.433083,1.220678,0.133787,100.123508,0.066582,19.4348,6.058751,13.376049,4.529532,0.30,1.358860
